# Naive Bayes Spam Classifier

Naive Bayes classifiers are a type of machine learning algorithm based applying [bayes theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) with strong (naive) independence assumptions between features. In short, a naive bayes classifier treats every features independent from each other, making inference very efficient. These types of classifiers are commonly used for spam detection.

The TF-IDF Tabu list was inspired by this [kaggle post](https://www.kaggle.com/clydewang/a-naive-bayes-way-for-spam-classification/notebook), with some improvements such as using pandas functions to clean the entire dataset instead of loop through it, and creating a better train and test split of the data.

## Before we start...

Let's quickly cover some of the basic definitions needed to understand our problem.

### Bayes Theorem

Bayes Theorem describes the probability of an event, based on prior knowledge of conditions that might be related to the event. Bayes Theorem is considered "naive" because it assumes that the presence (or absence) of a particular feature of a class is unrelated to the presence (or absence) of any other feature. In other words, every feature is taken into account without considering the existence of another feature.

Mathmatically, Bayes Theorem can be written as:

$$
P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)}
$$

Lets break this down:
- $A$ and $B$ are considered seperate events, and the Probability of $B$ (ie $P(B)$) ≠ 0.
- $P(A)$ and $P(B)$ are the probabilities of observing events $A$ and $B$ without regard to each other.
- $ P(A \mid B) $ is the probability of observing event $A$ given that $B$ is true
- $ P(B \mid A) $ is the probability of observing event $B$ given that $A$ is true

When applying Bayes Theorem to spam classification, we can rewrite the problem statment as:

$$
P(\textrm{spam} \mid \textrm{w}1 \cap \textrm{w}2 \> \cap .. \cap \> \textrm{w}n) = \frac{P(\textrm{w}1 \cap \textrm{w}2 \> \cap \> .. \cap \> \textrm{w}n \mid \textrm{spam}) \, P(\textrm{spam})}{P(\textrm{w}1 \cap \textrm{w}2 \> \cap \> .. \cap \> \textrm{w}n)}
$$

Now, we have a message m that is made up of n number of words, or m = $ (w1 \cap w2 \cap .. \cap wn) $. We assume the occurence of any word wn is independent of all other words.



## 1. Environment Setup

In [1]:
# Loads watermark extension and prints details about current platform
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,matplotlib
# autoreloads changes in imported files
%load_ext autoreload
%autoreload 2
 
# import packages
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
from sklearn.naive_bayes import BernoulliNB

# Get project directory
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
print(PROJ_ROOT)
import sys

# make sure matplotlib will display inline
%matplotlib inline

Fri Dec 28 2018 

CPython 3.7.1
IPython 7.2.0

numpy 1.15.4
scipy 1.1.0
sklearn 0.20.1
pandas 0.23.4
matplotlib 3.0.2

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 18.0.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
/Users/sebp/LocalDocuments2/DataScience/Personal/MachineLearning/MachineLearning


## 2. Read in Data

We'll define a function that reads in our data and splits it into a training and test dataset.

In [2]:
def read_data():
    SMS_df = pd.read_csv(PROJ_ROOT +'/data/naive_bayes/raw/spam.csv',usecols=[0,1],encoding='latin-1')
    SMS_df.columns=['label','content']
    n = int(SMS_df.shape[0])
    # split into training data and test data
    return split_train_test(SMS_df)

def split_train_test(df, train_size=0.8):
    """ Splits data into train and test dataframes. Defaults to 80 20 split if not specified"""
    split_df = pd.DataFrame(np.random.randn(df.shape[0], 2))
    msk = np.random.rand(len(df)) < train_size
    train = df[msk]
    test = df[~msk]
    return train, test

## 3. Generate a Tabu List
A **tabu list** is a list of those significant indicators of spam SMS. Here we select TF-IDF as the principle of list generation.

**Term Frequency Inverse Document Frequence (TF-IDF)** is a statistical representation of the most important words in a collection of documents, or collection of messages in our case. The TF-IDF value increases proportionally the more often a word is found in a document, and is offset by the number of times that word occurs in the collection.

Term Frequency (TF) is the frequency of a word in a certain kind of document. If there is a article of 50 words with 2 'data' in it, then the TF of the 'data' is given by 2/50=0.01.

However, there are some words of high frequency in English, like 'a', 'is', 'are', etc. We have to remove those words from our list. We'll use IDF (inverse document frequency) to acomplish this task. Inverse Document Frequency is the indicator to reflect how important a word is related to some certain topic. It is given by  

$$
\textrm{IDF} = \log(\frac{\textrm{total articles number of articles}}{\textrm{total number of articles containing word w}})
$$

The more common a word is across all examples, the lower it's "importance" score will be. Each word gets assigned a score, which is it's Term Frequency x Inverse Document Frequency (TF(w) * IDF(w)).

Here is the code:

In [3]:
def generate_tabu_list(path, tabu_size=200,ignore=3):
    """
    path = file name to use for exporting list
    tabu_size = length of exported list (ie how many rows through dataframe that function will process)
    ignore = minimum word length necessary to process word (ie ignore common short words like a, at, the, etc)
    """
    train_df,_ = read_data()
    spam_TF_dict = dict()
    valid_TF_dict = dict()
    IDF_dict = dict()

    # ignore all other than letters.
    # returns list of words downcased, removing punctuation and anything that is not a letter
    train_df['cleaned_content'] = train_df.content.apply( lambda x: [i.lower() for i in re.findall('[A-Za-z]+', re.sub("'","",x))])
    
#   # go through each word in the dataset and add it to a dict of 
    for i in range(train_df.shape[0]):
        if train_df.iloc[i].label == 'spam':
            for find in train_df.iloc[i].cleaned_content:
                if len(find)<ignore: continue
                try:
                    # if the current word is already in our spam dict, increment the value (ie number of
                    # occurences) by 1
                    spam_TF_dict[find] = spam_TF_dict[find] + 1
                except:	
                    # if the current word is not in our spam dict, add it, set the initial value to 1
                    # and add the word to our valid dict and set the value to 0
                    spam_TF_dict[find] = spam_TF_dict.get(find,1)
                    valid_TF_dict[find] = valid_TF_dict.get(find,0)
        else:
            for find in train_df.iloc[i].cleaned_content:
                if len(find)<ignore: continue
                try:
                    valid_TF_dict[find] = valid_TF_dict[find] + 1
                except:	
                    spam_TF_dict[find] = spam_TF_dict.get(find,0)
                    valid_TF_dict[find] = valid_TF_dict.get(find,1)

        # basically just a list of each unique word
        word_set = set()
        for find in train_df.iloc[i].cleaned_content:
            if len(find)<ignore: continue
            if not(find in word_set):
                try:
                    IDF_dict[find] = IDF_dict[find] + 1
                except:	
                    IDF_dict[find] = IDF_dict.get(find,1)
            word_set.add(find)

    word_df = pd.DataFrame(list(zip(valid_TF_dict.keys(),valid_TF_dict.values(),spam_TF_dict.values(),IDF_dict.values())))
    word_df.columns = ['keyword','valid_TF','spam_TF','IDF']
    word_df['valid_TF'] = word_df['valid_TF'].astype('float')/train_df[train_df['label']=='ham'].shape[0]
    word_df['spam_TF'] = word_df['spam_TF'].astype('float')/train_df[train_df['label']=='spam'].shape[0]
    word_df['IDF'] = np.log10(train_df.shape[0]/word_df['IDF'].astype('float'))
    word_df['valid_TFIDF'] = word_df['valid_TF']*word_df['IDF']
    word_df['spam_TFIDF'] = word_df['spam_TF']*word_df['IDF']
    word_df['diff']=word_df['spam_TFIDF']-word_df['valid_TFIDF']

    selected_spam_key = word_df.sort_values('diff',ascending=False)

    print('>>>Generating Tabu List...\n  Tabu List Size: {}\n  File Name: {}\n  The words shorter than {} are ignored by model\n'.format(tabu_size, path, ignore))
    file = open(path,'w')
    for word in selected_spam_key.head(tabu_size).keyword:
        file.write(word+'\n')
    file.close()

## 4. Read Tabu List and Convert SMS
Since the message is of variant length, it is not easy for the implementation of learning algorithm. So we define a Function above generating tabu list and storing them in the local file. And we can use this file to convert a SMS expressed in string to a vector of fixed length expressed in binary value.

The idea is given like this: If we have a tabu list then we could find those word in the list and represent them by a index. Thus a string can be converted to an array of int. Further, we could define an array filled with zeros with the same length of tabu list. if this str contains the word in the tabu list, we could assign 1 to the corresponding element of the array representing 'message contains word w'. (tips: the query of python.dict is of constant time, much faster than python.list)

By taking this step, we could convert our raw data of variant length into the numeric data of fixed length.

These two function is given below:

In [4]:
def read_tabu_list(path):
    file = open(path,'r')
    keyword_dict = dict()
    i = 0
    for line in file:
        keyword_dict.update({line.strip():i})
        i+=1
    return keyword_dict

# create a numpy array of length tabu, ie the number of unique words
# go through each word passed in 'content' (content is a string of words)
# for each unique word in the string, find it's index in the numpy array,
# and set its value to '1' to show it exists
def convert_content(content, tabu):
    m = len(tabu)
    res = np.int_(np.zeros(m))
    finds = re.findall('[A-Za-z]+', content)
    for find in finds:
        find=find.lower()
        try:
            i = tabu[find]
            res[i]=1
        except:
            continue
    return res

## 5. Learning, Testing and Predicting
After we generate our tabu list and those supporting functions, we are now well prepared for the learning part in this problem. And here we could use the library from sklearn.naive_bayes import BernoulliNB. It will help us train this model.

Before this part, let review our data: our feature input X is a nm matrix, where X[i,j] = 1 means the sample #i contains the word j in the tabu list, and supervised label Y is a n1 vector where Y[i] = 1 representing for a spam and 0 for a ham.

Let prepare the materials for the learning algorithm.

In [5]:
def learn():
    global tabu, m
    train,_ = read_data()
    n = train.shape[0]
    X = np.zeros((n,m)); Y=np.int_(train.label=='spam')
    for i in range(n):
        X[i,:] = convert_content(train.iloc[i].content, tabu)

    NaiveBayes = BernoulliNB()
    NaiveBayes.fit(X, Y)

    Y_hat = NaiveBayes.predict(X)
    print('>>>Learning...\n  Learning Sample Size: {}\n  Accuarcy (Training sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
    return NaiveBayes

The Function above returns a well trained Naive Bayes Model object, and we could use it to make prediction.

In [6]:
def test(NaiveBayes):
    global tabu, m
    _,test = read_data()
    n = test.shape[0]
    X = np.zeros((n,m)); Y=np.int_(test.label=='spam')
    for i in range(n):
        X[i,:] = convert_content(test.iloc[i].content, tabu)
    Y_hat = NaiveBayes.predict(X)
    print ('>>>Cross Validation...\n  Testing Sample Size: {}\n  Accuarcy (Testing sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
    return

def predictSMS(SMS):
    global NaiveBayes, tabu, m
    X = convert_content(SMS, tabu)
    Y_hat = NaiveBayes.predict(X.reshape(1,-1))
    if int(Y_hat) == 1:
        print ('SPAM: {}'.format(SMS))
    else:
        print ('HAM: {}'.format(SMS))

## 6. Overall Assembly
After we define the every modules we need in this problem, we could integrate them into a whole part.

In [7]:
print('UCI SMS SPAM CLASSIFICATION PROBLEM SET\n  -- implemented by Bernoulli Naive Bayes Model\n')
tabu_file = PROJ_ROOT + '/data/naive_bayes/interim/tabu.txt'          # user defined tabu file
tabu_size = 300                 # how many features are used to classify spam
word_len_ignored = 3            # ignore those words shorter than this variable
# build a tabu list based on the training data
generate_tabu_list(tabu_file,tabu_size, word_len_ignored)

tabu = read_tabu_list(tabu_file)
m = len(tabu)
# train the Naive Bayes Model using training data
NaiveBayes=learn()
# Test Model using testing data
test(NaiveBayes)
print('>>>Testing')
# I select two messages from the test data here.
predictSMS('Ya very nice. . .be ready on thursday')
predictSMS('Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call')

UCI SMS SPAM CLASSIFICATION PROBLEM SET
  -- implemented by Bernoulli Naive Bayes Model

>>>Generating Tabu List...
  Tabu List Size: 300
  File Name: /Users/sebp/LocalDocuments2/DataScience/Personal/MachineLearning/MachineLearning/data/naive_bayes/interim/tabu.txt
  The words shorter than 3 are ignored by model

>>>Learning...
  Learning Sample Size: 4473
  Accuarcy (Training sample): 98.10％

>>>Cross Validation...
  Testing Sample Size: 1143
  Accuarcy (Testing sample): 98.69％

>>>Testing
HAM: Ya very nice. . .be ready on thursday
SPAM: Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call
